In [198]:
import numpy as np 
import tensorflow as tf
import tensorflow.compat.v1 as tf1
import pandas as pd
import matplotlib.pyplot as plt
import math
from  tensorflow.python.framework import ops


In [199]:
tf.compat.v1.disable_eager_execution()

In [200]:
import h5py

In [201]:
from tensorflow.keras.datasets import mnist

In [202]:
 (x_train, y_train), (x_test, y_test) = mnist.load_data()

In [203]:
x_train.shape

(60000, 28, 28)

In [204]:
x_train = x_train.reshape( (x_train.shape[0],x_train.shape[1]*x_train.shape[2]) )/255

In [205]:
x_train.shape

(60000, 784)

In [235]:
y_train[5]

array([2], dtype=uint8)

In [206]:
x_test = x_test.reshape( (x_test.shape[0], x_test.shape[1] * x_test.shape[2]) )/255

In [207]:
x_test.shape

(10000, 784)

In [208]:
y_train = y_train.reshape(y_train.shape[0],1)

In [209]:
def create_placeholder(n_x, n_y):
    X = tf1.placeholder(shape = [None,n_x], dtype = tf.float32, name = 'X')
    y = tf1.placeholder(shape = [None,n_y], dtype = tf.float32, name = 'y')
    return X,y

In [210]:
X , y = create_placeholder((28*28),1)

In [211]:
def init_params(n_units):
    tf1.set_random_seed(1)
    l = len(n_units)
    params = {}
    for i in range(1,l):
        params["w"+str(i)] = tf1.get_variable("w"+str(i),[n_units[i] , n_units[i-1]],\
                                             initializer= tf.initializers.glorot_uniform(seed=1))
        params["b"+str(i)] = tf1.get_variable("b"+str(i),[n_units[i] , 1],initializer= tf.initializers.Zeros())
        
    return params
        

In [212]:
tf1.reset_default_graph()
with tf1.Session() as sess:
    init_params([21,43,53])

In [213]:
def forward_prop(X, params):
    l = len(params)//2
    A={}
    Z={}
    A[0]= X
    for i in range(1,l):
        w = params.get("w"+str(i))
        b = params.get("b"+str(i))
        Z[i] = tf.add(tf.matmul (w,A[i-1],transpose_a=False) ,b)
        A[i] = tf.nn.relu(Z[i])
        
    w = params.get("w"+str(l))
    b = params.get("b"+str(l))
    Z[l] = tf.add(tf.matmul (w,A[l-1], transpose_a=False) ,b)
    return Z[l]
        
        

In [214]:
def compute_cost(Zl, y):
    logits = tf.transpose(Zl)
    labels = tf.transpose(y)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels= labels, logits= logits))
    return cost

In [218]:
def random_mini_bathces(x, y, batch_size,seed):
    m = x.shape[0]
    mini_batches = []
    np.random.seed(seed)
    permutation = list(np.random.permutation(m))
    shuffled_x = x[permutation,:]
    shuffled_y = y[permutation,:].reshape((m,y.shape[1]))
    num_complete_minibatches = math.floor(m/batch_size)
    for k in range(0,num_complete_minibatches):
        batch_x = shuffled_x[ k*batch_size:k*batch_size+batch_size,:]
        batch_y = shuffled_y[ k*batch_size:k*batch_size+batch_size,:]
        mini_batches.append((batch_x,batch_y))
    if m % batch_size != 0 :
        batch_x = shuffled_x[ num_complete_minibatches*batch_size:m,:]
        batch_y = shuffled_y[ num_complete_minibatches*batch_size:m,:]
        mini_batches.append((batch_x,batch_y))
    return mini_batches

In [236]:
def model(x, y, x_test, y_test, learning_rate= 0.0001, epochs=1500, batch_size= 32, print_cost=True ):
    ops.reset_default_graph()
    tf1.set_random_seed(1)
    seed = 3
    (m, n_x) = x.shape
    n_y = y.shape[1]
    costs = []
    
    X , Y = create_placeholder(n_x,n_y)
    params = init_params([(28*28),64,32,32,10,1])
    Zl = forward_prop(X, params)
    cost = compute_cost(Zl, Y)
    optimizer = tf1.train.AdamOptimizer(learning_rate= learning_rate).minimize(cost)
    init = tf1.global_variables_initializer()
    with tf1.Session() as  sess:
        sess.run(init)
        
        for epoch in range(epochs):
            epoch_cost = 0
            num_batches = int (m /batch_size)
            seed +=1
            batches = random_mini_bathces(x, y, batch_size,seed)
            
            for batch in batches:
                batch_x, batch_y = batch
                _,batch_cost = sess.run([optimizer,cost], feed_dict={X:batch_x,Y:batch_y})
                
                epoch_cost+= batch_cost / num_batches
            
            if print_cost and epoch % 100 == 0 :
                print(f'cost after epoch {epoch} : {epoch_cost}')
            if print_cost and epoch % 5 == 0 :
                costs.append(epoch_cost)
        
        plt.plot(np.squeeze(costs))
        plt.ylabel('cost')
        plt.xlabel('iter per fives')
        plt.show()
        
        params = sess.run(params)
        correct_pred = tf.equal(tf.argmax(Zl),tf.argmax(Y))
        acc = tf.reduce_mean(tf.cast(correct_pred,'float'))
        
        print(f"train acc :",acc.eval({X:x,Y:y}))
        print(f"test acc :",acc.eval({X:x_test,Y:y_test}))
        
        return params

                

In [237]:
params = model(x_train,y_train,x_test,y_test)

ValueError: Incompatible shapes between op input and calculated input gradient. Forward operation: softmax_cross_entropy_with_logits. Input index: 1. Original input shape: (1, ?). Calculated input gradient shape: (784, 1)